### Model Point File (MPF) Data Validation Check

#### Load the Model Point File and the defined rules

In [1]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# load inputs
df_rules = pd.read_excel(r'MPF_Data_Validation_Check_Sample.xlsx', sheet_name="Rules_Input")
df_mpf_check1 = pd.read_excel(r'MPF_Data_Validation_Check_Sample.xlsx', sheet_name="MPF_Input_Completeness_Check")
df_mpf = pd.read_excel(r'MPF_Data_Validation_Check_Sample.xlsx', sheet_name="MPF_Input")

#### Check 1: Completeness Check for all the columns

In [2]:
def completeness_check(df: pd.DataFrame):
    """If any column contains NaN, blank, or empty values, it returns an error message with the affected columns."""
    # Identify columns with NaN, blanks, or empty strings
    incomplete_columns = [col for col in df.columns if df[col].isna().any() or (df[col].astype(str).str.strip() == '').any()]
    
    if incomplete_columns:
        return f"Error: The following columns contain NaN, blank, or empty values: {', '.join(incomplete_columns)}"
    else:
        return "All columns are complete."

In [3]:
completeness_check(df_mpf_check1)

'Error: The following columns contain NaN, blank, or empty values: sum_assured_dth, sum_assured_tpd, sum_assured_trm, Occupation, Benefit Period Type'

#### Check 2: Column "Policy number" check

In [4]:
def policy_number_check(df: pd.DataFrame):
    if 'Policy number' not in df.columns:
        return "Error: 'Policy number' column is missing."
    
    # Check if 'Policy number' contains only integers
    if not df['Policy number'].apply(lambda x: isinstance(x, int)).all():
        return "Error: 'Policy number' column must contain only integers."
    
    # Find duplicate policy numbers
    duplicates = df['Policy number'][df['Policy number'].duplicated(keep=False)].unique()
    
    if duplicates.any():
        return f"Error: Policy Number must be unique. Duplicates found: {', '.join(map(str, duplicates))}"
    
    return "All policy numbers are valid and unique."

In [5]:
policy_number_check(df_mpf)

'Error: Policy Number must be unique. Duplicates found: 19'

#### Check 3: Column "policy_term" check

In [6]:
def policy_term_check(df):
    # Identify rows where policy_term is not an integer greater than 0
    invalid_rows = df[(df["policy_term"] <= 0) | (df["policy_term"] % 1 != 0)]
    
    if not invalid_rows.empty:
        print("Error: The following policy numbers have incorrect policy terms")
        return invalid_rows[["Policy number", "policy_term"]].reset_index(drop=True)
    else:
        print("All policy terms are correct.")
        return None

In [7]:
policy_term_check(df_mpf)

Error: The following policy numbers have incorrect policy terms


,Policy number,policy_term
0,11,0


#### Check 4: Check all the numeric amount columns

In [8]:
def numeric_amt_check(df):
    columns_to_check = ["sum_assured_dth", "sum_assured_tpd", "sum_assured_trm", "Annual Prem", 
                         "R_sum_assured_dth", "R_sum_assured_tpd", "R_sum_assured_trm", "R_Prem", 
                         "Monthly Benefit", "R_Monthly_Ben"]
    
    # Ensure all values are numeric and non-negative
    invalid_rows = df[~df[columns_to_check].map(lambda x: isinstance(x, (int, float)) and x >= 0).all(axis=1)]
    
    # Ensure sum_assured_dth, Annual Prem, and Monthly Benefit are greater than zero
    mandatory_columns = ["sum_assured_dth", "Annual Prem", "Monthly Benefit"]
    invalid_mandatory_rows = df[df[mandatory_columns].map(lambda x: x == 0).any(axis=1)]
    
    invalid_rows = pd.concat([invalid_rows, invalid_mandatory_rows]).drop_duplicates()
    
    if not invalid_rows.empty:
        print("Error: The following policy numbers have incorrect numeric amount values")
        return invalid_rows[["Policy number"] + columns_to_check].reset_index(drop=True)
    else:
        print("All numeric amount values are correct.")
        return None

In [9]:
numeric_amt_check(df_mpf)

Error: The following policy numbers have incorrect numeric amount values


,Policy number,sum_assured_dth,sum_assured_tpd,sum_assured_trm,Annual Prem,R_sum_assured_dth,R_sum_assured_tpd,R_sum_assured_trm,R_Prem,Monthly Benefit,R_Monthly_Ben
0,4,-1000.5,0,0,1600,0,0,0,50,2817,1408.5
1,10,0.0,0,0,1600,0,0,0,50,2817,1408.5
2,11,100000.0,0,0,0,0,0,0,0,0,1408.5
3,12,100000.0,0,0,0,0,0,0,0,0,1408.5
4,13,100000.0,0,0,0,0,0,0,0,2817,1408.5


#### Check 5: Column "DOB" check

In [10]:
def dob_check(df):
    today = datetime.today().date()
    min_date = today - timedelta(days=65*365)  # 65 years ago
    max_date = today - timedelta(days=18*365)  # 18 years ago
    
    # Ensure DOB is a string, clean spaces, and convert to datetime
    df["DOB"] = df["DOB"].astype(str).str.strip()
    
    try:
        df["DOB"] = pd.to_datetime(df["DOB"], format="%Y-%m-%d", errors="coerce").dt.date
    except Exception as e:
        print("Error converting DOB:", e)
    
    invalid_rows = df[(df["DOB"].isna()) | (df["DOB"] < min_date) | (df["DOB"] > max_date)]
    
    if not invalid_rows.empty:
        print("Error: The following policy numbers have incorrect DOB values")
        return invalid_rows[["Policy number", "DOB"]].reset_index(drop=True)
    else:
        print("All DOB values are correct.")
        return None

In [11]:
dob_check(df_mpf)

Error: The following policy numbers have incorrect DOB values


,Policy number,DOB
0,11,2018-05-01
1,16,2013-05-01


#### Check 6: Column "Entry date" check

In [12]:
def entry_date_check(df, val_date):
    val_date = datetime.strptime(val_date, "%Y-%m-%d").date()
    
    df["Entry date"] = pd.to_datetime(df["Entry date"], errors="coerce").dt.date
    
    invalid_rows = df[(df["Entry date"].isna()) | (df["Entry date"] < df["DOB"]) | (df["Entry date"] > val_date)]
    
    if not invalid_rows.empty:
        print("Error: The following policy numbers have incorrect entry dates")
        return invalid_rows[["Policy number", "Entry date"]].reset_index(drop=True)
    else:
        print("All entry dates are correct.")
        return None

In [13]:
entry_date_check(df_mpf, "2024-06-30")

Error: The following policy numbers have incorrect entry dates


,Policy number,Entry date
0,12,1990-11-11
1,16,2024-11-10


#### Check 7: Check against all defined rules

In [14]:
def generic_check(df, df_rules, column_name):
    # Extract the valid values from df_rules and convert them into a set
    valid_values = set(df_rules.loc[df_rules["Column"] == column_name, "Input_Array"].str.split(", ").explode())
    
    # Convert column to string for consistent comparison
    df[column_name] = df[column_name].astype(str)
    
    # Identify rows with invalid values
    invalid_rows = df[~df[column_name].isin(valid_values)]
    
    if not invalid_rows.empty:
        print(f"{column_name} Check:")
        print(f"Error: The following policy numbers have incorrect {column_name} values")
        print(invalid_rows[["Policy number", column_name]].reset_index(drop=True))
    else:
        print(f"{column_name} Check:")
        print(f"All {column_name} values are correct.")

def integer_check(df, columns):
    for column in columns:
        invalid_rows = df[(~df[column].apply(lambda x: isinstance(x, int))) | (df[column] <= 0)]
        
        if not invalid_rows.empty:
            print(f"{column} Check:")
            print(f"Error: The following policy numbers have incorrect {column} values")
            print(invalid_rows[["Policy Number", column]].reset_index(drop=True))
        else:
            print(f"All {column} values are valid integers greater than zero.")

# List of columns to check
columns_to_check = ["Product", "sex", "pols_if_init", "Prem Freq", "Prem_Increase_ind", "Smoker status", "Stepped_ind", "Occupation", 
                    "Waiting Period", "Benefit Period Type", "Benefit Period", "Benefit Type", "Prem Waiver"]

integer_columns = ["IFRS17_Contract_Boundary", "IFRS17_Rein_Contract_Boundary", "Related_Policy_Group", "Related_Policy_Group_Rein"]

In [15]:
def check_all_columns(df, df_rules):
    for column in columns_to_check:
        generic_check(df, df_rules, column)
    integer_check(df, integer_columns)

In [16]:
check_all_columns(df_mpf, df_rules)

Product Check:
Error: The following policy numbers have incorrect Product values
   Policy number Product
0              9    D_IP
1             12  C_TREM
sex Check:
Error: The following policy numbers have incorrect sex values
   Policy number     sex
0              1    Male
1             11  Female
pols_if_init Check:
Error: The following policy numbers have incorrect pols_if_init values
   Policy number pols_if_init
0             11            Y
Prem Freq Check:
Error: The following policy numbers have incorrect Prem Freq values
   Policy number Prem Freq
0             17         6
1             18         6
2             19        26
Prem_Increase_ind Check:
Error: The following policy numbers have incorrect Prem_Increase_ind values
   Policy number Prem_Increase_ind
0             17                 Y
1             18                 N
Smoker status Check:
Error: The following policy numbers have incorrect Smoker status values
   Policy number Smoker status
0             11      